In [1]:
import model
import fastText
from keras.callbacks import ModelCheckpoint
from importlib import reload
import sklearn.metrics as sklm
import numpy as np

Using TensorFlow backend.


In [2]:
# from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [4]:
reload(model)

<module 'model' from '/srv/home/gwiedemann/pss-lre/model.py'>

In [5]:
data_train = model.read_csv_data("data/Tobacco800/dataset.train", csvformat="Tobacco800")
data_test = model.read_csv_data("data/Tobacco800/dataset.test", csvformat="Tobacco800")

In [7]:
if 'ft' not in locals():
    ft = fastText.load_model("../embeddings/wiki.en.bin")
    model.ft = ft

# Experiment: Simple CNN model (single page)
* optimize for Kappa
* 10 repeats

In [11]:
n_repeats = 10
n_epochs = 20
exp_history = []
optimize_for = 'kappa'
for i in range(n_repeats):
    print("Repeat " + str(i+1) + " of " + str(n_repeats))
    print("-------------------------")
    model_singlepage = model.compile_model_singlepage()
    model_file = "tobacco800_exp1_single-page_repeat-%02d.hdf5" % (i,)
    print(model_file)
    checkpoint = model.ValidationCheckpoint(model_file, data_test, metric=optimize_for)
    model_singlepage.fit_generator(generator=model.TextFeatureGenerator(data_train),
                    callbacks = [checkpoint],
                    epochs = n_epochs)
    exp_history.append(checkpoint.max_metrics)

avg_result = sum([m['kappa'] for m in exp_history]) / n_repeats
avg_acc = sum([m['accuracy'] for m in exp_history]) / n_repeats
print("-------------------------")
print(avg_result)
print(avg_acc)

Repeat 1 of 10
-------------------------
tobacco800_exp1_single-page_repeat-00.hdf5
Epoch 1/20
33/33 [==============================] - 18s 549ms/step - loss: 0.8519 - acc: 0.5988

kappa improvement: 0.3543265761333071 (before: -inf), saving to tobacco800_exp1_single-page_repeat-00.hdf5
Epoch 2/20
33/33 [==============================] - 15s 443ms/step - loss: 0.5984 - acc: 0.7011
Epoch 3/20
33/33 [==============================] - 15s 454ms/step - loss: 0.5367 - acc: 0.7427

kappa improvement: 0.535126113927638 (before: 0.3543265761333071), saving to tobacco800_exp1_single-page_repeat-00.hdf5
Epoch 4/20
33/33 [==============================] - 15s 448ms/step - loss: 0.4775 - acc: 0.7835
Epoch 5/20
33/33 [==============================] - 15s 447ms/step - loss: 0.3908 - acc: 0.8257
Epoch 6/20
33/33 [==============================] - 15s 463ms/step - loss: 0.3876 - acc: 0.8275
Epoch 7/20
33/33 [==============================] - 14s 433ms/step - loss: 0.3907 - acc: 0.8394

kappa improvem

Repeat 4 of 10
-------------------------
tobacco800_exp1_single-page_repeat-03.hdf5
Epoch 1/20
33/33 [==============================] - 18s 560ms/step - loss: 0.9333 - acc: 0.6111

kappa improvement: 0.38884076585322136 (before: -inf), saving to tobacco800_exp1_single-page_repeat-03.hdf5
Epoch 2/20
33/33 [==============================] - 15s 444ms/step - loss: 0.6187 - acc: 0.6830
Epoch 3/20
33/33 [==============================] - 15s 444ms/step - loss: 0.5727 - acc: 0.7295
Epoch 4/20
33/33 [==============================] - 15s 461ms/step - loss: 0.4803 - acc: 0.7561

kappa improvement: 0.49317985611510795 (before: 0.38884076585322136), saving to tobacco800_exp1_single-page_repeat-03.hdf5
Epoch 5/20
33/33 [==============================] - 15s 451ms/step - loss: 0.4026 - acc: 0.8342
Epoch 6/20
33/33 [==============================] - 15s 453ms/step - loss: 0.4228 - acc: 0.8224

kappa improvement: 0.5485183033120278 (before: 0.49317985611510795), saving to tobacco800_exp1_single-page

Epoch 11/20
33/33 [==============================] - 15s 468ms/step - loss: 0.2769 - acc: 0.8958
Epoch 12/20
33/33 [==============================] - 16s 471ms/step - loss: 0.2601 - acc: 0.8906
Epoch 13/20
33/33 [==============================] - 15s 448ms/step - loss: 0.2365 - acc: 0.9043
Epoch 14/20
33/33 [==============================] - 15s 453ms/step - loss: 0.2089 - acc: 0.9143
Epoch 15/20
33/33 [==============================] - 15s 450ms/step - loss: 0.1728 - acc: 0.9251

kappa improvement: 0.6922584772872681 (before: 0.6823839597768104), saving to tobacco800_exp1_single-page_repeat-08.hdf5
Epoch 16/20
33/33 [==============================] - 15s 458ms/step - loss: 0.1949 - acc: 0.9205
Epoch 17/20
33/33 [==============================] - 16s 470ms/step - loss: 0.1463 - acc: 0.9488
Epoch 18/20
33/33 [==============================] - 15s 454ms/step - loss: 0.1379 - acc: 0.9427

kappa improvement: 0.7307033639143731 (before: 0.6922584772872681), saving to tobacco800_exp1_single-

In [12]:
# average: 0.6867630405833453
for i, r in enumerate(exp_history):
    model_file = "tobacco800_exp1_single-page_repeat-%02d.hdf5" % (i,)
    print(str(i) + ' ' + str(r['kappa']) + ' ' + str(r['accuracy']) + ' ' + str(r['f1_macro']) + ' ' + model_file)

0 0.7094916179100859 0.8571428571428571 0.8546378460371634 tobacco800_exp1_single-page_repeat-00.hdf5
1 0.7119733102681329 0.861003861003861 0.8558441558441559 tobacco800_exp1_single-page_repeat-01.hdf5
2 0.692071835304424 0.8532818532818532 0.845251572327044 tobacco800_exp1_single-page_repeat-02.hdf5
3 0.7119733102681329 0.861003861003861 0.8558441558441559 tobacco800_exp1_single-page_repeat-03.hdf5
4 0.6723845370684602 0.8416988416988417 0.8360683575960604 tobacco800_exp1_single-page_repeat-04.hdf5
5 0.6828160936706592 0.8416988416988417 0.8405483730498371 tobacco800_exp1_single-page_repeat-05.hdf5
6 0.6572703492845053 0.833976833976834 0.8285692295851612 tobacco800_exp1_single-page_repeat-06.hdf5
7 0.6867809854569134 0.8494208494208494 0.8430961368190504 tobacco800_exp1_single-page_repeat-07.hdf5
8 0.7307033639143731 0.8687258687258688 0.8653516819571866 tobacco800_exp1_single-page_repeat-08.hdf5
9 0.6434515987735436 0.8301158301158301 0.820817610062893 tobacco800_exp1_single-page_r

# Experiment: Predecessor page

In [13]:
n_repeats = 10
n_epochs = 20
exp1b_history = []
optimize_for = 'kappa'
for i in range(n_repeats):
    print("Repeat " + str(i+1) + " of " + str(n_repeats))
    print("-------------------------")
    model_prevpage = model.compile_model_prevpage()
    model_file = "tobacco800_exp1_prev-page_repeat-%02d.hdf5" % (i,)
    print(model_file)
    checkpoint = model.ValidationCheckpoint(model_file, data_test, prev_page_generator = True, metric=optimize_for)
    model_prevpage.fit_generator(generator=model.TextFeatureGenerator2(data_train),
                    callbacks = [checkpoint],
                    epochs = n_epochs)
    exp1b_history.append(checkpoint.max_metrics)

avg_result = sum([m['kappa'] for m in exp1b_history]) / n_repeats
print("-------------------------")
print(avg_result)

Repeat 1 of 10
-------------------------
tobacco800_exp1_prev-page_repeat-00.hdf5
Epoch 1/20
33/33 [==============================] - 35s 1s/step - loss: 0.8948 - acc: 0.5817

kappa improvement: 0.010611205432937143 (before: -inf), saving to tobacco800_exp1_prev-page_repeat-00.hdf5
Epoch 2/20
33/33 [==============================] - 27s 819ms/step - loss: 0.6528 - acc: 0.6585

kappa improvement: 0.2801166022642534 (before: 0.010611205432937143), saving to tobacco800_exp1_prev-page_repeat-00.hdf5
Epoch 3/20
33/33 [==============================] - 27s 828ms/step - loss: 0.5629 - acc: 0.7233

kappa improvement: 0.35860764072456097 (before: 0.2801166022642534), saving to tobacco800_exp1_prev-page_repeat-00.hdf5
Epoch 4/20
33/33 [==============================] - 27s 821ms/step - loss: 0.4957 - acc: 0.7726

kappa improvement: 0.42848203405460594 (before: 0.35860764072456097), saving to tobacco800_exp1_prev-page_repeat-00.hdf5
Epoch 5/20
33/33 [==============================] - 26s 783ms/st

/srv/home/gwiedemann/miniconda3/envs/nnnlp/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



kappa improvement: 0.0 (before: -inf), saving to tobacco800_exp1_prev-page_repeat-01.hdf5
Epoch 2/20
33/33 [==============================] - 27s 812ms/step - loss: 0.6360 - acc: 0.6623

kappa improvement: 0.4617622610141314 (before: 0.0), saving to tobacco800_exp1_prev-page_repeat-01.hdf5
Epoch 3/20
33/33 [==============================] - 28s 836ms/step - loss: 0.5857 - acc: 0.7086
Epoch 4/20
33/33 [==============================] - 27s 815ms/step - loss: 0.5177 - acc: 0.7570

kappa improvement: 0.5755490003277615 (before: 0.4617622610141314), saving to tobacco800_exp1_prev-page_repeat-01.hdf5
Epoch 5/20
33/33 [==============================] - 26s 792ms/step - loss: 0.4325 - acc: 0.8058
Epoch 6/20
33/33 [==============================] - 26s 800ms/step - loss: 0.3941 - acc: 0.8337

kappa improvement: 0.6272448532632502 (before: 0.5755490003277615), saving to tobacco800_exp1_prev-page_repeat-01.hdf5
Epoch 7/20
33/33 [==============================] - 27s 812ms/step - loss: 0.3530 - 

33/33 [==============================] - 28s 842ms/step - loss: 0.1290 - acc: 0.9526
Epoch 18/20
33/33 [==============================] - 27s 813ms/step - loss: 0.1136 - acc: 0.9583
Epoch 19/20
33/33 [==============================] - 27s 821ms/step - loss: 0.1097 - acc: 0.9630
Epoch 20/20
33/33 [==============================] - 27s 833ms/step - loss: 0.1139 - acc: 0.9593
Repeat 8 of 10
-------------------------
tobacco800_exp1_prev-page_repeat-07.hdf5
Epoch 1/20
33/33 [==============================] - 39s 1s/step - loss: 0.9484 - acc: 0.5206

kappa improvement: 0.0 (before: -inf), saving to tobacco800_exp1_prev-page_repeat-07.hdf5
Epoch 2/20
33/33 [==============================] - 27s 817ms/step - loss: 0.6580 - acc: 0.6510

kappa improvement: 0.38036533535984685 (before: 0.0), saving to tobacco800_exp1_prev-page_repeat-07.hdf5
Epoch 3/20
33/33 [==============================] - 28s 847ms/step - loss: 0.5772 - acc: 0.7262

kappa improvement: 0.44078775189740904 (before: 0.380365335

33/33 [==============================] - 27s 809ms/step - loss: 0.2107 - acc: 0.9166
Epoch 15/20
33/33 [==============================] - 27s 828ms/step - loss: 0.1756 - acc: 0.9237
Epoch 16/20
33/33 [==============================] - 27s 810ms/step - loss: 0.1285 - acc: 0.9479
Epoch 17/20
33/33 [==============================] - 27s 813ms/step - loss: 0.1809 - acc: 0.9346
Epoch 18/20
33/33 [==============================] - 28s 857ms/step - loss: 0.1497 - acc: 0.9446
Epoch 19/20
33/33 [==============================] - 27s 823ms/step - loss: 0.1956 - acc: 0.9285
Epoch 20/20
33/33 [==============================] - 28s 843ms/step - loss: 0.1597 - acc: 0.9351
-------------------------
0.6929258620318568


In [14]:
avg_kappa = sum([m['kappa'] for m in exp1b_history]) / n_repeats
avg_acc = sum([m['accuracy'] for m in exp1b_history]) / n_repeats
print("-------------------------")
print(avg_kappa)
print(avg_acc)

-------------------------
0.6929258620318568
0.8509652509652508


In [15]:
for i, r in enumerate(exp1b_history):
    model_file = "tobacco800_exp1_prev-page_repeat-%02d.hdf5" % (i,)
    print(str(i) + ' ' + str(r['kappa']) + ' ' + str(r['accuracy']) + ' ' + model_file)

0 0.6891521772580397 0.8494208494208494 tobacco800_exp1_prev-page_repeat-00.hdf5
1 0.6546559583242891 0.833976833976834 tobacco800_exp1_prev-page_repeat-01.hdf5
2 0.7134251290877797 0.861003861003861 tobacco800_exp1_prev-page_repeat-02.hdf5
3 0.6616791354945969 0.8301158301158301 tobacco800_exp1_prev-page_repeat-03.hdf5
4 0.7279747930310145 0.8687258687258688 tobacco800_exp1_prev-page_repeat-04.hdf5
5 0.7065739918552314 0.8571428571428571 tobacco800_exp1_prev-page_repeat-05.hdf5
6 0.7300263658102888 0.8687258687258688 tobacco800_exp1_prev-page_repeat-06.hdf5
7 0.6681873574352405 0.8416988416988417 tobacco800_exp1_prev-page_repeat-07.hdf5
8 0.6951997522452772 0.8532818532818532 tobacco800_exp1_prev-page_repeat-08.hdf5
9 0.6823839597768104 0.8455598455598455 tobacco800_exp1_prev-page_repeat-09.hdf5


In [16]:
# load best model
model_prevpage.load_weights("tobacco800_exp1_prev-page_repeat-06.hdf5")
y_predict = np.round(model_prevpage.predict_generator(model.TextFeatureGenerator2(data_test, batch_size=256)))
y_true = [model.label2Idx[x[1]] for x in data_test]
print("Accuracy: " + str(sklm.accuracy_score(y_true, y_predict)))
print("Kappa: " + str(sklm.cohen_kappa_score(y_true, y_predict)))

Accuracy: 0.8687258687258688
Kappa: 0.7300263658102888
